In [1]:
%env OPENAI_API_KEY = api_key

env: OPENAI_API_KEY=api_key


In [2]:
# pip install --upgrade openai

In [3]:
import os
import json
import re
import time
import subprocess
import tiktoken

In [4]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL = 'gpt-3.5-turbo'
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [5]:
def get_refined_error_log(stderr):
    # Modify stderr to only include the .java, not the full file path
    refined_stderr = '\n'.join([line.split('/')[-1] if '.java' in line else line for line in stderr.split('\n')])

    return refined_stderr

In [6]:
def extract_code(input_string):
    pattern = r"(package|import|@[\w]+|public|private|protected).*\}\s*$"
    match = re.search(pattern, input_string, re.DOTALL | re.MULTILINE)
    
    if match:
        return match.group(0)  # get the entire matched string
    else:
        print("No valid code block found!")
        return "No valid code block found!"

In [7]:
def get_fixed_code(java_code, refined_stderr, attempt, chat_history, chat_history_code):
    message_content = ""
    main_prompt = "Now fix the error only by fixing the import statements by not using wildcard imports and must not modify code body which means do not change anything inside the class. So, it can be successfully compiled and reply with full code."
    if attempt < 2:
        message_content = f"See the code below:\n\"\"\"\n{java_code}\n\"\"\"\nFor the above code I got the below error log:\n\"\"\"\n{refined_stderr}\n\"\"\"\n{main_prompt}"
        if len(encoding.encode(message_content)) > 4000:
            message_content = f"{java_code}\n\"\"\"\n{main_prompt}"
            if len(encoding.encode(message_content)) > 4000:
                return "0", message_content
            
    else:
        message_content = f"{chat_history}{main_prompt}"
        if len(encoding.encode(message_content)) > 4000:
            if chat_history_code != "":
                message_content = f"{chat_history_code}\n\"\"\"\n{main_prompt}"
                if len(encoding.encode(message_content)) > 4000:
                    return "0"
            else:
                return "0", message_content
            
    retry_delay = 2
    while True:
        try:
            response = openai.ChatCompletion.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "Reply with only code, no elaboration."},
                    {"role": "user", "content": f"{message_content}"},
                ],
                temperature=0.5,
            )
            
            return extract_code(response["choices"][0]["message"]["content"]), message_content
        
        except Exception as e:
            time.sleep(retry_delay)

In [8]:
chat_history_error_message = ""

def compile_java(attempt, used_prompt, new_java_code_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path):
    try:
        is_compile_success = False
        # If the output directory doesn't exist, create it
        if not os.path.exists(compiled_folder_path):
            os.makedirs(compiled_folder_path)
        
        # Compile the .java file with the provided class_path and specify the output directory for .class files
        result = subprocess.run(['javac', '-cp', class_path, '-d', compiled_folder_path, new_java_code_path], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print(f"Attempt {attempt}. Successfully compiled \"{new_java_code_path}\".")
            stderr_log = result.stderr
            is_compile_success = True
        else:
            warning_val = False
            num_errors = 1
            stderr_lines = result.stderr.splitlines()
            num_errors_line = stderr_lines[-1]
            try:
                num_errors = int(num_errors_line.split()[0])  # Extract the number of error(s)
            except ValueError:
                warning_val = True

            r_errors_count = result.stderr.count("error: package R does not exist")

            # Check if all errors are related to "package R does not exist"
            if num_errors == r_errors_count and warning_val == False:
                print(f"Attempt {attempt}. Successfully compiled, ignoring {num_errors} 'package R does not exist' errors for \"{new_java_code_path}\".")
                stderr_log = "No error (Ignored 'package R does not exist' errors)"
                is_compile_success = True
            else:
                if warning_val:
                    print(f"Attempt {attempt}. Compiled with warning.")
                    stderr_log = "Warning present"
                    is_compile_success = True
                else:
                    print(f"Attempt {attempt}. Failed to compile \"{new_java_code_path}\".")
                    stderr_log = result.stderr
                    global chat_history_error_message
                    chat_history_error_message = '\n'.join([line.split('/')[-1] if '.java' in line else line for line in result.stderr.split('\n')])
                    is_compile_success = False

        # Read the content of the Java file
        with open(new_java_code_path, 'r') as java_file:
            java_code = java_file.read()
        # Prepare the JSON log
        log_data = {
            "attempt": attempt,
            "file": new_java_code_path,
            "java_code": java_code,
            "used_prompt": used_prompt,
            "stdout": result.stdout,
            "stderr": stderr_log,
            "return_code": result.returncode
        }

        if is_compile_success or (not is_compile_success and attempt == max_attempt):
        
            # Save the logs to a json file named after the Java file being compiled
            base_name = os.path.basename(new_java_code_path).replace('.java', '')
            log_file_name = f"{base_name}_attempt_{attempt}.json"
            if is_compile_success:
                log_file_path = os.path.join(log_folder_success, log_file_name)
            else:
                log_file_path = os.path.join(log_folder_fail, log_file_name)
            
            stringified_data = {str(key): value for key, value in log_data.items()}
            with open(log_file_path, 'w') as log_file:
                json.dump(stringified_data, log_file, indent=4)
        else:
            # Save the logs to a txt file named after the Java file being compiled
            base_name = os.path.basename(new_java_code_path).replace('.java', '')
            log_file_name = f"{base_name}_attempt_{attempt}.txt"
            
            log_file_path = os.path.join(log_folder_fail, log_file_name)
            
            with open(log_file_path, 'w') as log_file:
                for key, value in log_data.items():
                    if key == "java_code" or key == "used_prompt":
                        if isinstance(value, str):
                            log_file.write(f"\"{key}\": \"\n\n{value}\n\"\n\n")
                        else:
                            log_file.write(f"\"{key}\": {value}\n\n")
                    else:
                        if isinstance(value, str):
                            log_file.write(f"\"{key}\": \"{value}\"\n\n")
                        else:
                            log_file.write(f"\"{key}\": {value}\n\n")

        return is_compile_success

    except subprocess.TimeoutExpired:
        print(f"Compilation of {new_java_code_path} timed out.")
        return False

In [9]:
def compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path):
    java_code_path = data["file"]
    java_code = data["java_code"]
    refined_stderr = get_refined_error_log(data["stderr"])
    
    if not os.path.exists(log_folder_success):
        os.makedirs(log_folder_success)

    if not os.path.exists(log_folder_fail):
        os.makedirs(log_folder_fail)
    
    # Maximum tries to get a successful compilation
    chat_history = ""
    chat_history_code = ""
    fixed_code = ""
    global chat_history_error_message
    chat_history_error_message = ""
                           
    for i in range(max_attempt):
        attempt = i+1
        if i==0:
            fixed_code, used_prompt = get_fixed_code(java_code, refined_stderr, attempt, chat_history, chat_history_code)
        else:
            # Keeping the last fixed code by the model inside java_code
            java_code = fixed_code
            chat_history = f"{chat_history}{chat_history_code}\n\nYou gave the above imports fix in your attempt {i}. But compiler gave this error:\n\n{chat_history_error_message}\n\n"
            if len(encoding.encode(chat_history)) > 4000:
                last_chat_history = f"{chat_history_code}\n\nYou gave the above imports fix. But compiler gave this error:\n\n{chat_history_error_message}\n\n"
                fixed_code, used_prompt = get_fixed_code(java_code, refined_stderr, attempt, last_chat_history, chat_history_code)
            else:
                fixed_code, used_prompt = get_fixed_code(java_code, refined_stderr, attempt, chat_history, chat_history_code)
        
        if fixed_code == "0":
            fixed_code = java_code
            
        chat_history_code = fixed_code
        # Saving the predicted fixed code by the model
        new_java_file_name = java_code_path.split('/')[-1]
        new_java_code_path = os.path.join(fixed_path, new_java_file_name)
        with open(new_java_code_path, 'w') as file:
            file.write(fixed_code)
        if compile_java(attempt, used_prompt, new_java_code_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path):
            return 1

    return 0

In [10]:
# Zero-shot Self-consistency from 10 Sample 5 Attempts

max_attempt = 5
dir_type = "zero-shot"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/{dir_type}-fix/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/{dir_type}-compiled/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/gwt_class_2.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/gwt_class_2.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/gwt_class_2.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/gwt_class_2.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-fix/xstream_class_40.java".
Attempt 2. Faile

In [10]:
# Zero-shot Self-consistency from 10 Sample 10 Attempts

max_attempt = 10
dir_type = "10-attempt"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/compile_fail"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-{dir_type}/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/zero-shot-{dir_type}/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt/gwt_class_2.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt/gwt_class_2.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt/gwt_class_2.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt/gwt_class_2.java".
Attempt 6. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-10-attempt

In [11]:
# Zero-shot Self-consistency from 10 Sample 15 Attempts

max_attempt = 15
dir_type = "15-attempt"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/compile_fail"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-{dir_type}/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/zero-shot-{dir_type}/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt/gwt_class_2.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt/gwt_class_2.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt/gwt_class_2.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt/gwt_class_2.java".
Attempt 6. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-15-attempt

In [10]:
# Zero-shot Self-consistency from 20 Sample 5 Attempts

max_attempt = 5
dir_type = "from-20-sample"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_fail/"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-{dir_type}/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/zero-shot-{dir_type}/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-20-sample/gwt_class_8.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-20-sample/Android42.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-20-sample/Android42.java".
Attempt 3. Successfully compiled, ignoring 6 'package R does not exist' errors for "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-20-sample/Android42.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-20-sample/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_

In [11]:
# Zero-shot Self-consistency from 30 Sample 5 Attempts

max_attempt = 5
dir_type = "from-30-sample"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/zero-shot-logs/{dir_type}/compile_fail/"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-{dir_type}/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/zero-shot-{dir_type}/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/zero-shot-{dir_type}/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-30-sample/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-30-sample/gwt_class_2.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-30-sample/gwt_class_2.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-30-sample/gwt_class_2.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/zero-shot-from-30-sample/gwt_class_2.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes

In [11]:
# One-shot Self-consistency with Semantic Similarity from 10 Sample 5 Attempts

max_attempt = 5
dir_type = "one-shot"

existing_failed_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail"
fixed_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/{dir_type}-fix/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/{dir_type}-compiled/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/gwt_class_2.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/gwt_class_2.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/gwt_class_2.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/gwt_class_2.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/gwt_class_2.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/one-shot-fix/hibernate_class_13.java".
Attempt 2. Failed to

In [11]:
# For StatType-SO

max_attempt = 5

existing_failed_folder_path = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_logs/compile_fail/"
fixed_path = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/"

jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/so-logs/compile_success/"
log_folder_fail = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-logs/so-logs/compile_fail/"
compiled_folder_path = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/llm-compiled-classes/so-compiled/" 

successful_compilations = 0
for filename in os.listdir(existing_failed_folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(existing_failed_folder_path, filename), "r") as f:
            data = json.load(f)
            success_count = compile_code_and_result(data, fixed_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
            successful_compilations += success_count
            print()

total_files = len([f for f in os.listdir(existing_failed_folder_path) if f.endswith(".json")])
success_rate = (successful_compilations / total_files) * 100

print(f"Compilation success rate: {success_rate:.2f}%")
print(f"Number of successfully compiled files: {successful_compilations} out of {total_files}.")

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Android02.java".
Attempt 2. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Android02.java".
Attempt 3. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Android02.java".
Attempt 4. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Android02.java".
Attempt 5. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Android02.java".

Attempt 1. Successfully compiled "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/llm_fixes/fixed_codes/so-fix/Class_19.java".

Attempt 1. Failed to compile "/home/azmain/GitHub Codes/Type_Inference_